#   MACHINE LEARNING

El modelo fue entrenado en local y el Deploy se hizo con Streamlit conectando con Github donde se encontraba el .PKL que es el modelo ya entrenado.

#   Entrenamiento del modelo

Este modelo se entrena con ElasticNet que es un método de regularización utilizado en el aprendizaje automático y la estadística, especialmente en problemas de regresión.

In [ ]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

def get_borough(location_id):
            # Agrupamos los PULocationID y DOLocationID de acuerdo a su Borough correspondiente
    zones = {   #Manhattan
                1 : [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 103, 103, 107, 113, 114, 116, 120, 125,
                            127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211,
                            224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263],

                #Brooklyn
                2 : [11, 14, 17, 21, 22, 25, 26, 29, 33, 34, 35, 36, 37, 39, 40, 49, 52, 54, 55, 61, 62, 63, 65, 66, 67, 71, 72, 76, 77, 80,
                            85, 89, 91, 97, 106, 108, 111, 112, 123, 133, 149, 150, 154, 155, 165, 177, 178, 181, 188, 189, 190, 195, 210, 217, 222,
                            225, 227, 228, 255, 256, 257],

                #Queens
                3 : [2, 7, 8, 9, 10, 15, 16, 19, 27, 28, 30, 38, 53, 56, 56, 64, 70, 73, 82, 83, 86, 92, 93, 95, 96, 98, 101, 102, 117, 121,
                        122, 124, 129, 130, 131, 132, 134, 135, 138, 139, 145, 146, 157, 160, 171, 173, 175, 179, 180, 191, 192, 193, 196, 197,
                        198, 201, 203, 205, 207, 215, 216, 218, 219, 223, 226, 252, 253, 258, 260],

                #Bronx
                4 : [3, 18, 20, 31, 32, 46, 47, 51, 58, 59, 60, 69, 78, 81, 94, 119, 126, 136, 147, 159, 167, 168, 169, 174, 182, 183, 184,
                        185, 199, 200, 208, 212, 213, 220, 235, 240, 241, 242, 247, 248, 250, 254, 259],
                #State Island
                5 : [5, 6, 23, 44, 84, 99, 109, 110, 115, 118, 156, 172, 176, 187, 204, 206, 214, 221, 245, 251],

                #EWR
               6 : [1]}

    for borough, ids in zones.items():
        if location_id in ids:
            return borough
    return 7

meses = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
df_meses = []

for mes in meses:
    df_mes = pd.read_parquet(f'yellow_tripdata_2022-{mes}.parquet')
    df_mes = df_mes.sample(n=500, random_state=42).dropna()
    df_mes["PickupZone"] = df_mes["PULocationID"].apply(get_borough)
    df_mes['tpep_pickup_datetime'] = df_mes['tpep_pickup_datetime'].dt.hour
    df_meses.append(df_mes)

df = pd.concat(df_meses)
X = df[["tpep_pickup_datetime", 'trip_distance', 'PickupZone']]
y = df['total_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Utiliza ElasticNet en lugar de RandomForestRegressor
model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Guarda el modelo entrenado para su uso en Streamlit
joblib.dump(model, 'ModeloEntrenado.pkl')

Este codigo se llama Stream.py y es el que estructura el Streamlit, se puede ver que se pide la Hora, las millas y la zona de donde se toma.

Omito los requirements.txt

In [ ]:
import streamlit as st
import pandas as pd
import joblib
import requests
from io import BytesIO
import sklearn


# Función para cargar el modelo desde GitHub
def load_model():
    # URL raw del archivo .pkl en GitHub
    model_url = 'https://github.com/darksider10/PF/raw/main/ML/ModeloEntrenado.pkl'
    
    # Descargar el archivo .pkl desde la URL
    response = requests.get(model_url)
    
    # Guardar el archivo .pkl localmente
    with open('ModeloEntrenado.pkl', 'wb') as f:
        f.write(response.content)
    
    # Cargar el modelo desde el archivo descargado
    model = joblib.load('ModeloEntrenado.pkl')
    return model

model = load_model()

# Creando la interfaz de usuario
st.title('Predicción del Total Amount para viajes de Taxi')

tpep_pickup_datetime= st.number_input('Hora', min_value=0, max_value=23)
trip_distance = st.number_input('Distancia del viaje (millas)', min_value=0.0, format="%.2f")
pickup_zone = st.selectbox('Zona de recogida', options=[('6','EWR'),('5', 'Staten Island'),('3', 'Queens'),('4', 'Bronx'), ('-1', 'Desconocido'),('2', 'Brooklyn'),('1', 'Manhattan')], format_func=lambda x: x[1])

# Botón para realizar la predicción
if st.button('Predecir Total Amount'):
    X_new = pd.DataFrame([[tpep_pickup_datetime, trip_distance, int(pickup_zone[0])]], columns=['tpep_pickup_datetime', 'trip_distance', 'PickupZone'])
    y_pred_new = model.predict(X_new)
    st.success(f'La predicción del total_amount es: ${y_pred_new[0]:.2f}')